In [14]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


We define our graph as an igraph.Graph object. Python igraph is a library for high-performance graph generation and analysis. Install the Python library with sudo pip install python-igraph.

Get graph data:

In [15]:
import igraph as ig
import pickle
import helpers
import plots_config
import numpy as np

full_links_dict = pickle.load(open('data/billionairesdict.pkl','rb'))
links_dict = helpers.trim_dict(full_links_dict, 150)
data = helpers.dict_to_nodes(links_dict)


Get the number of nodes:

In [16]:
N=len(data['nodes'])
N

150

Define the list of edges and the Graph object from Edges:

In [17]:
L=len(data['links'])
Edges=[(data['links'][k]['source'], data['links'][k]['target']) for k in range(L)]

G=ig.Graph(Edges, directed=False)

Extract the node attributes, 'group', and 'name':

In [18]:
data['nodes'][0]

{'name': 'Elon Musk', 'group': 4279914}

In [19]:
labels=[]
group=[]
for node in data['nodes']:
    labels.append(node['name'] + ' | Links to this page: ' + str(len(links_dict[node['name']]['linkshere_within_category'])))
    group.append(node['group'])

Get the node positions, set by the Kamada-Kawai layout for 3D graphs:

In [20]:
layt=G.layout('kk', dim=3)
layt.scale(100)
len(layt)

150

layt is a list of three elements lists (the coordinates of nodes):

In [21]:
layt[0]

[-55.75683663386775, 76.25895838948244, 261.8611831673949]

Set data for the Plotly plot of the graph:

In [22]:
Xn=[layt[k][0] for k in range(len(layt))]# x-coordinates of nodes
Yn=[layt[k][1] for k in range(len(layt))]# y-coordinates
Zn=[layt[k][2] for k in range(len(layt))]# z-coordinates
Xe=[]
Ye=[]
Ze=[]
for e in Edges:
    Xe+=[layt[e[0]][0],layt[e[1]][0], None]# x-coordinates of edge ends
    Ye+=[layt[e[0]][1],layt[e[1]][1], None]
    Ze+=[layt[e[0]][2],layt[e[1]][2], None]

In [28]:
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot

trace1=go.Scatter3d(x=Xe,
               y=Ye,
               z=Ze,
               mode='lines',
               line=dict(color='rgb(180,155,191)', width=1),
               hoverinfo='none'
               )

trace2=go.Scatter3d(x=Xn,
               y=Yn,
               z=Zn,
               mode='markers',
               name='billionaires',
               marker=dict(symbol='circle',
                             size=6,
                             color=np.log10(group),
                            #  colorscale='Viridis',
                            autocolorscale=True,
                             line=dict(color='rgb(50,50,50)', width=0.5)
                             ),
               text=labels,
               hoverinfo='text'
               )

axis = plots_config.kk_axis_config

layout = go.Layout(
         title="Links between 150 most viewed American billionaires' Wikipedia pages",
         title_font_color='rgb(255,255,255)',
         width=1000,
         height=800,
         showlegend=False,
         scene=dict(
             xaxis=dict(axis),
             yaxis=dict(axis),
             zaxis=dict(axis),
             bgcolor='rgb(22,16,25)'
        ), paper_bgcolor='rgb(22,16,25)',
     margin=dict(
        t=50
    ),
    annotations=[
           dict(
           showarrow=False,
            text='Note that warmer colors indicate a larger number of page views',
            xref='paper',
            yref='paper',
            x=0,
            y=0.1,
            xanchor='left',
            yanchor='bottom',
            font=dict(
            size=14,
            color='rgb(255,255,255)'
            )
            )
        ],
    hovermode='closest'
    )


In [27]:
data=[trace1, trace2]
fig=go.Figure(data=data, layout=layout)

iplot(fig)


In [12]:
#fig.write_html("plots/3dwide.html")